In [1]:
import os
import sys
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
sys.path.insert(1, os.getenv("PROJECT_ROOT"))
os.environ['HF_HOME'] = os.getenv("HF_CACHE")

from transformers import AutoTokenizer, AutoModel
import torch
import pickle
from tqdm import tqdm
import torch.nn as nn
import dgl
from dgl import heterograph
from collections import defaultdict
import numpy as np
import networkx as nx
from pathlib import Path

# Load SciBERT model
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased", local_files_only=True)
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased", local_files_only=True).to(device)


/raid/infolab/sarvam/env_sd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_scibert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
    return cls_embedding  # shape: (1, hidden_size)


paper_records = pickle.load(open('data/teacher_graph/records/paper_records.pkl', "rb"))
author_records = pickle.load(open('data/teacher_graph/records/author_records.pkl', "rb"))
fields_of_study = set()
affiliations = set()

all_ids = set(paper_records.keys())
student_paper_ids = pickle.load(open('data/teacher_graph/paper_ids.pkl', "rb"))
eval_ids = pickle.load(open('data/teacher_graph/eval_ids.pickle', "rb"))
test_ids = pickle.load(open('data/teacher_graph/test_ids.pickle', "rb"))
extra_train_ids = pickle.load(open('data/teacher_graph/extra_train_ids.pickle', "rb"))

# create a combined set of student paper ids and extra train ids
train_ids = set(student_paper_ids).union(extra_train_ids)

# combined_ids = set(train_ids).union(eval_ids)

train_ids_nums = {i:idx for idx, i in enumerate(train_ids)}
train_nodes = [v for k,v in train_ids_nums.items()]

eval_ids_nums = {i:idx for idx, i in enumerate(eval_ids)}
eval_nodes = [v for k,v in eval_ids_nums.items()]

# for author_id, author in tqdm(author_records.items()):
#     affiliations.update(author['affiliations'])

# for s2_id, paper in tqdm(paper_records.items()):
#     fields_of_study.update(paper['field_ids'])



train_graph = nx.DiGraph()

# add nodes for all papers
train_graph.add_nodes_from(train_nodes)

# add edges for all papers
edges = []

for s2_id in tqdm(train_ids):
    paper = paper_records[s2_id]
    for ref_id in paper['reference_ids']:
        if ref_id in train_ids:
            if ref_id == None:
                continue
            edges.append((train_ids_nums[s2_id], train_ids_nums[ref_id])) # paper -> reference
   
train_graph.add_edges_from(edges)
train_feats = torch.zeros((len(train_ids), 768))


eval_edges = []
for s2_id in tqdm(eval_ids):
    paper = paper_records[s2_id]
    for ref_id in paper['reference_ids']:
        if ref_id in train_ids:
            if ref_id == None:
                continue
            eval_edges.append((eval_ids_nums[s2_id], train_ids_nums[ref_id])) # paper -> reference


# Compute and assign SciBERT embeddings for paper nodes
for local_idx, global_id in tqdm(enumerate(train_ids)):
    pdata = paper_records[global_id]
    if pdata['abstract'] is None:
        abstract = ""
    else:
        abstract = pdata['abstract']
    text = pdata['title'] + "\n" + abstract + "\n"
    text += f"This paper was published in {pdata['venue']} in {pdata['year']}. It has {len(pdata["author_ids"])} authors and {pdata['referenceCount']} references. It has {pdata["citationCount"]} citations and {pdata["influentialCitationCount"]} influential citations."
    emb = get_scibert_embedding(text)
    train_feats[local_idx] = emb

# 1. Prepare eval node SciBERT embeddings (once)
eval_feats = torch.zeros(len(eval_ids), 768)
for local_idx, global_id in tqdm(enumerate(eval_ids)):
    pdata = paper_records[global_id]
    if pdata['abstract'] is None:
        abstract = ""
    else:
        abstract = pdata['abstract']
    text = pdata['title'] + "\n" + abstract + "\n"
    emb = get_scibert_embedding(text)
    eval_feats[local_idx] = emb

# train the model using the graph attention network
import torch.nn.functional as F
from torch_geometric.nn import GATConv

train_feats = train_feats.to(device)
eval_feats = eval_feats.to(device)

combined_feats = torch.cat((train_feats, eval_feats), dim=0)
combined_feats = combined_feats.to(device)

100%|██████████| 1500/1500 [00:00<00:00, 75110.20it/s]
11528it [01:26, 133.05it/s]
1500it [00:10, 139.35it/s]


In [3]:
eval_graph = nx.DiGraph()
# add nodes for all papers
eval_graph.add_nodes_from(eval_nodes)
# add edges for all papers
eval_graph.add_edges_from([])

In [4]:
def compute_loss(pos_score, neg_score):
    pos_loss = -F.logsigmoid(pos_score).mean()
    neg_loss = -F.logsigmoid(-neg_score).mean()
    return pos_loss + neg_loss

In [5]:
def calculate_recall_at_k_fixed(eval_ids, predicted_indices, k):
    """
    Calculate recall@k for evaluation nodes, mapping indices back to global IDs.
    Args:
        eval_ids: List of evaluation node IDs (global IDs)
        predicted_indices: Tensor of predicted train node indices for each eval node
        k: Number of top predictions to consider
        train_ids_list: List of train node IDs (global IDs) in the same order as used in the model
    Returns:
        recall: Recall@k value
    """
    relevant_count = 0
    total_relevant = 0
    
    for eval_idx, eval_id in enumerate(eval_ids):
        paper = paper_records[eval_id]
        true_references = set(ref for ref in paper['reference_ids'] if ref in train_ids)
        true_references = set([train_ids_nums[i] for i in true_references])
        total_relevant += len(true_references)
        
        # Map predicted indices back to global IDs
        predicted_refs = predicted_indices[eval_idx, :k].cpu().tolist()
        # predicted_refs = set(train_ids_list[idx] for idx in pred_indices)
        
        relevant_count += len(true_references.intersection(predicted_refs))
    
    if total_relevant == 0:
        return 0.0
    
    return relevant_count / total_relevant

in_dim = train_feats.shape[1]
hidden_dim = 128
num_heads = 4

class DotLinkPredictor(nn.Module):
    def forward(self, h, src_idx, dst_idx):
        return (h[src_idx] * h[dst_idx]).sum(dim=-1)
    
    # Alternative implementation that avoids the in-place operation issue
class GATLinkPredictorFixed(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_heads):
        super(GATLinkPredictorFixed, self).__init__()
        self.gat1 = GATConv(in_dim, hidden_dim, num_heads)
        self.gat2 = GATConv(hidden_dim * num_heads, hidden_dim, 1)
        
    # def encode_without_graph(self, features):
    #     """Process nodes without graph structure (for evaluation)"""
    #     h = self.gat1(features, edge_index=None)
    #     h = F.elu(h.flatten(1))
    #     h = self.gat2(h, edge_index=None).squeeze(1)
    #     return h
    
    def forward(self, g, features):
        # For NetworkX graph compatibility
        if isinstance(g, nx.DiGraph):
            edge_index = torch.tensor(list(g.edges())).t().to(device)
            # create empty tensor of size (2, num_edges)
            if edge_index.shape[0] == 0:
                edge_index = torch.zeros((2, len(eval_edges)), dtype=torch.long).to(device)
            h = self.gat1(features, edge_index)
            h = F.elu(h.flatten(1))
            h = self.gat2(h, edge_index).squeeze(1)
        else:
            # Original implementation for other graph types
            h = self.gat1(g, features)
            h = F.elu(h.flatten(1))
            h = self.gat2(g, h).squeeze(1)
        return h

# Fixed training loop
model_fixed = GATLinkPredictorFixed(in_dim, hidden_dim, num_heads).to(device)
predictor = DotLinkPredictor()
optimizer = torch.optim.Adam(list(model_fixed.parameters()) + list(predictor.parameters()), lr=1e-3)

for epoch in range(40):
    model_fixed.train()
    
    # Get edges for training
    edge_list = list(train_graph.edges())
    src, dst = zip(*edge_list)
    src = torch.tensor(src, dtype=torch.long).to(device)
    dst = torch.tensor(dst, dtype=torch.long).to(device)
    n_edges = src.shape[0]
    
    # Process all edges in a single batch with multiple epochs
    # This prevents in-place modification issues
    batch_losses = []
    
    # Process in smaller sub-epochs
    for sub_epoch in range(5):  # 5 sub-epochs per epoch
        perm = torch.randperm(n_edges)
        
        for i in tqdm(range(0, n_edges, 1024)):
            # Get embeddings for all nodes
            h = model_fixed(train_graph, train_feats)
            
            batch_src = src[perm[i:i+1024]]
            batch_dst = dst[perm[i:i+1024]]
            
            # Negative sampling
            neg_dst = torch.randint(0, h.shape[0], batch_dst.shape, dtype=torch.long).to(device)
            
            pos_score = predictor(h, batch_src, batch_dst)
            neg_score = predictor(h, batch_src, neg_dst)
            
            loss = compute_loss(pos_score, neg_score)
            batch_losses.append(loss.item())
            
            optimizer.zero_grad()
            loss.backward()  # No retain_graph needed
            optimizer.step()
    
    avg_loss = sum(batch_losses) / len(batch_losses)
    print(f"Epoch {epoch} | Average Loss: {avg_loss:.4f}")
    
    # Evaluation
    model_fixed.eval()
    with torch.no_grad():
        # Get train node embeddings
        train_embs = model_fixed(train_graph, train_feats)
        
        # Get eval node embeddings without graph structure
        eval_embs = model_fixed(eval_graph, eval_feats)
        
        # Compute similarity scores
        scores = torch.matmul(eval_embs, train_embs.T)
        
        # Get top-k predictions
        topk = torch.topk(scores, k=20, dim=1)
        predicted_indices = topk.indices
        
        # Calculate recall@k with the fixed function
        recall_at_10 = calculate_recall_at_k_fixed(eval_ids, predicted_indices, 10)
        recall_at_20 = calculate_recall_at_k_fixed(eval_ids, predicted_indices, 20)
        
        print(f"Recall@10: {recall_at_10:.4f} | Recall@20: {recall_at_20:.4f}")

100%|██████████| 63/63 [00:03<00:00, 17.87it/s]


Epoch 0 | Average Loss: 1.9430
Recall@10: 0.0012 | Recall@20: 0.0022


100%|██████████| 63/63 [00:03<00:00, 20.56it/s]


Epoch 1 | Average Loss: 1.0357
Recall@10: 0.0016 | Recall@20: 0.0035


100%|██████████| 63/63 [00:04<00:00, 12.74it/s]


...

Epoch 37 | Average Loss: 0.8802
Recall@10: 0.0639 | Recall@20: 0.0869


100%|██████████| 63/63 [00:04<00:00, 13.75it/s]


Epoch 38 | Average Loss: 0.8790
Recall@10: 0.0685 | Recall@20: 0.0965


100%|██████████| 63/63 [00:04<00:00, 14.49it/s]


Epoch 39 | Average Loss: 0.8769
Recall@10: 0.0552 | Recall@20: 0.0772


In [11]:
torch.save(model_fixed.state_dict(), 'model/gat_model_plus.pt')

In [12]:
model2 = GATLinkPredictorFixed(in_dim, hidden_dim, num_heads)
model2.load_state_dict(torch.load('model/gat_model_plus.pt'))
model2.to(device)  # move to GPU if needed
model2.eval()      # set to evaluation mode

/tmp/ipykernel_3151549/1001121453.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('model/gat_model_plus.pt'))


GATLinkPredictorFixed(
  (gat1): GATConv(768, 128, heads=4)
  (gat2): GATConv(512, 128, heads=1)
)